<a href="https://colab.research.google.com/github/sp7412/colab/blob/master/teacher_student.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook I use knowledge distillation on MNIST.
The goal is not to classify MNIST as well as possible, but rather show improvement thanks to distillation on a nano model.

Bigger model achivies just barely 99%.

Smaller model seems to improve around 0.3% depending on the run.

In [ ]:
import tensorflow as tf

In [ ]:
mnist = tf.keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
train,  test = mnist
x_train, y_train = train
x_test, y_test = train

x_train, x_test = x_train/255, x_test/255

In [ ]:
x_train.shape

(60000, 28, 28)

In [ ]:
def softmax_sparse_categorical_crossentropy(labels, logits):
  softmaxed = tf.keras.backend.softmax(logits)

  return tf.keras.losses.sparse_categorical_crossentropy(labels, softmaxed)

In [ ]:
control_model = tf.keras.models.Sequential([
  tf.keras.Input([28, 28]), 
  tf.keras.layers.Reshape([28, 28, 1]), 
  tf.keras.layers.Conv2D(64, 3, 2, activation='relu'), 
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Conv2D(64, 3, 2, activation='relu'), 
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling2D(), 
  #tf.keras.layers.Dense(10, activation='softmax')])
  tf.keras.layers.Dense(10, activation=None)])

#control_model.compile('adam', 'sparse_categorical_crossentropy', ['accuracy'])
control_model.compile('adam', softmax_sparse_categorical_crossentropy, ['accuracy'])
control_model.summary()

Model: "sequential_33"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_33 (Reshape)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_67 (Conv2D)           (None, 13, 13, 64)        640       
_________________________________________________________________
batch_normalization_7 (Batch (None, 13, 13, 64)        256       
_________________________________________________________________
dropout_3 (Dropout)          (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_68 (Conv2D)           (None, 6, 6, 64)          36928     
_________________________________________________________________
batch_normalization_8 (Batch (None, 6, 6, 64)          256       
_________________________________________________________________
dropout_4 (Dropout)          (None, 6, 6, 64)        

In [ ]:
control_model.fit(x_train, y_train, epochs=30, validation_split=0.2)

Epoch 1/30
1500/1500 [==============================] - 4s 3ms/step - loss: 0.7650 - accuracy: 0.7840 - val_loss: 0.3173 - val_accuracy: 0.9091
Epoch 2/30
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3200 - accuracy: 0.9090 - val_loss: 0.2435 - val_accuracy: 0.9283
Epoch 3/30
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2556 - accuracy: 0.9242 - val_loss: 0.1639 - val_accuracy: 0.9518
Epoch 4/30
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2215 - accuracy: 0.9323 - val_loss: 0.1691 - val_accuracy: 0.9504
Epoch 5/30
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2074 - accuracy: 0.9372 - val_loss: 0.1511 - val_accuracy: 0.9544
Epoch 6/30
1500/1500 [==============================] - 4s 3ms/step - loss: 0.1895 - accuracy: 0.9413 - val_loss: 0.1216 - val_accuracy: 0.9631
Epoch 7/30
1500/1500 [==============================] - 4s 3ms/step - loss: 0.1841 - accuracy: 0.9422 - val_loss: 0.1270 - val_accuracy:

In [ ]:
teacher_model = tf.keras.models.Sequential([
  tf.keras.Input([28, 28]), 
  tf.keras.layers.Reshape([28, 28, 1]), 
  tf.keras.layers.Conv2D(64, 3, 2, activation='relu'), 
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Conv2D(64, 3, 2, activation='relu'), 
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Conv2D(64, 3, 2, activation='relu'), 
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling2D(), 
  #tf.keras.layers.Dense(10, activation='softmax')])
  tf.keras.layers.Dense(10, activation=None)])

#teacher_model.compile('adam', 'sparse_categorical_crossentropy', ['accuracy'])
teacher_model.compile('adam', softmax_sparse_categorical_crossentropy, ['accuracy'])
teacher_model.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_16 (Reshape)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 13, 13, 64)        640       
_________________________________________________________________
batch_normalization (BatchNo (None, 13, 13, 64)        256       
_________________________________________________________________
dropout (Dropout)            (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 6, 6, 64)          36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 6, 6, 64)          256       
_________________________________________________________________
dropout_1 (Dropout)          (None, 6, 6, 64)        

In [ ]:
teacher_model.fit(x_train, y_train, epochs=30, validation_split=0.2)

Epoch 1/30
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3257 - accuracy: 0.9080 - val_loss: 0.0899 - val_accuracy: 0.9736
Epoch 2/30
1500/1500 [==============================] - 5s 3ms/step - loss: 0.1265 - accuracy: 0.9624 - val_loss: 0.0720 - val_accuracy: 0.9773
Epoch 3/30
1500/1500 [==============================] - 5s 3ms/step - loss: 0.0955 - accuracy: 0.9707 - val_loss: 0.0591 - val_accuracy: 0.9831
Epoch 4/30
1500/1500 [==============================] - 5s 3ms/step - loss: 0.0817 - accuracy: 0.9755 - val_loss: 0.0512 - val_accuracy: 0.9837
Epoch 5/30
1500/1500 [==============================] - 5s 3ms/step - loss: 0.0702 - accuracy: 0.9781 - val_loss: 0.0475 - val_accuracy: 0.9866
Epoch 6/30
1500/1500 [==============================] - 5s 3ms/step - loss: 0.0681 - accuracy: 0.9788 - val_loss: 0.0424 - val_accuracy: 0.9870
Epoch 7/30
1500/1500 [==============================] - 5s 3ms/step - loss: 0.0608 - accuracy: 0.9809 - val_loss: 0.0436 - val_accuracy:

In [ ]:
soft_labels = teacher_model.predict(x_train)

In [ ]:
temperature = 3
afterwards_temperature = 1

def temperature_softmax(logits):
  soft_logits = tf.keras.backend.exp(logits / temperature)
  return soft_logits / tf.keras.backend.sum(soft_logits, axis=-1, keepdims=True) / afterwards_temperature

def distillation_loss(labels, logits):
  labels = temperature_softmax(labels)
  logits = temperature_softmax(logits)

  return -tf.keras.backend.mean(labels * tf.keras.backend.log(logits))

In [ ]:
student_model = tf.keras.models.Sequential([
  tf.keras.Input([28, 28]), 
  tf.keras.layers.Reshape([28, 28, 1]), 
  tf.keras.layers.Conv2D(64, 3, 2, activation='relu'), 
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Conv2D(64, 3, 2, activation='relu'), 
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling2D(), 
  #tf.keras.layers.Dense(10, activation='softmax')])
  tf.keras.layers.Dense(10, activation=None)])

student_model.compile('adam', distillation_loss, ['accuracy'])
student_model.summary()

Model: "sequential_40"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_40 (Reshape)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_81 (Conv2D)           (None, 13, 13, 64)        640       
_________________________________________________________________
batch_normalization_21 (Batc (None, 13, 13, 64)        256       
_________________________________________________________________
dropout_17 (Dropout)         (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_82 (Conv2D)           (None, 6, 6, 64)          36928     
_________________________________________________________________
batch_normalization_22 (Batc (None, 6, 6, 64)          256       
_________________________________________________________________
dropout_18 (Dropout)         (None, 6, 6, 64)        

In [ ]:
student_model.fit(x_train, soft_labels, epochs=30, validation_split=0.2)

Epoch 1/30
1500/1500 [==============================] - 4s 3ms/step - loss: 0.0426 - accuracy: 0.9638 - val_loss: 0.0387 - val_accuracy: 0.9765
Epoch 2/30
1500/1500 [==============================] - 4s 3ms/step - loss: 0.0427 - accuracy: 0.9624 - val_loss: 0.0387 - val_accuracy: 0.9758
Epoch 3/30
1500/1500 [==============================] - 4s 3ms/step - loss: 0.0424 - accuracy: 0.9636 - val_loss: 0.0392 - val_accuracy: 0.9730
Epoch 4/30
1500/1500 [==============================] - 4s 3ms/step - loss: 0.0426 - accuracy: 0.9625 - val_loss: 0.0388 - val_accuracy: 0.9750
Epoch 5/30
1500/1500 [==============================] - 4s 3ms/step - loss: 0.0426 - accuracy: 0.9637 - val_loss: 0.0387 - val_accuracy: 0.9758
Epoch 6/30
1500/1500 [==============================] - 4s 3ms/step - loss: 0.0425 - accuracy: 0.9637 - val_loss: 0.0392 - val_accuracy: 0.9745
Epoch 7/30
1500/1500 [==============================] - 4s 3ms/step - loss: 0.0424 - accuracy: 0.9638 - val_loss: 0.0388 - val_accuracy: